# Code to produce the cutouts for the MIRI data

Let's start with the imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from miri_utils.cutout_tools import produce_cutouts


Let's look at the NIRCam documentation:

So we find that in the short filters we have 0.031"/pix and Amir used 167 x 167 pixels, which means he is using an image size of 5.177" for each cutout. We also know that the total FOV is around 63.74" x 63.74". Now we need to translate this to MIRI:

So taking this conversion of pixels to angular resolution we find that MIRI pixels are on average about 3.5 times larger than NIRCam pixels, so we just need to divide 167 by 3.55, which leaves us with ~48 pixels in MIRI.



# Produce cutouts while preserving all other layers (in the same patch of the sky)

# Create cutouts of the unmodified FITS files

Shifted data:

For the photometry we actually wanna use 8x8 arcsec cutouts for better noise statistics so that we increase the number of pixels from 48x48 to 74x74

In [ ]:
# Define directories
primer003 = '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/PRIMER_003_shifted/'
primer004 = '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/PRIMER_004_shifted/'

cweb1 = '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/COSMOS-Web_1_shifted/'
cweb2 = '/home/bpc/University/master/Red_Cardinal/MIRI_shifted/COSMOS-Web_2_shifted/'

cosmos3d_1 = '/home/bpc/University/master/Red_Cardinal/COSMOS-3D/COSMOS-3D_1'
cosmos3d_2 = '/home/bpc/University/master/Red_Cardinal/COSMOS-3D/COSMOS-3D_2'

catalogue =  '/home/bpc/University/master/Red_Cardinal/catalogues/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_phot/'
os.makedirs(output_dir, exist_ok=True)

# Produce cutouts for all surveys
#produce_cutouts(catalogue, primer003, output_dir, 'primer', 8, 'F770W', '003', nan_thresh=0.5)
#produce_cutouts(catalogue, primer003, output_dir, 'primer', 8, 'F1800W', '003', nan_thresh=0.5)

#produce_cutouts(catalogue, primer004, output_dir, 'primer', 8, 'F770W', '004', nan_thresh=0.5)
#produce_cutouts(catalogue, primer004, output_dir, 'primer', 8, 'F1800W', '004', nan_thresh=0.5)

#produce_cutouts(catalogue, cweb1, output_dir, 'cweb', 8, 'F770W', '1', nan_thresh=0.5)
#produce_cutouts(catalogue, cweb2, output_dir, 'cweb', 8, 'F770W', '2', nan_thresh=0.5)

produce_cutouts(catalogue, cosmos3d_1, output_dir, 'cos3d', 8, 'F1000W', '1', nan_thresh=0.5)
produce_cutouts(catalogue, cosmos3d_1, output_dir, 'cos3d', 8, 'F2100W', '1', nan_thresh=0.5)
produce_cutouts(catalogue, cosmos3d_2, output_dir, 'cos3d', 8, 'F1000W', '2', nan_thresh=0.5)
produce_cutouts(catalogue, cosmos3d_2, output_dir, 'cos3d', 8, 'F2100W', '2', nan_thresh=0.5)


# Rotated data

In [ ]:
# Define directories
cos3d1_rot = '/home/bpc/University/master/Red_Cardinal/COSMOS-3D/COSMOS-3D_1_rot'
cos3d2_rot = '/home/bpc/University/master/Red_Cardinal/COSMOS-3D/COSMOS-3D_2_rot'

catalogue =  '/home/bpc/University/master/Red_Cardinal/catalogues/cat_targets.fits'

output_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_3x3/'
os.makedirs(output_dir, exist_ok=True)

produce_cutouts(catalogue, cos3d1_rot, output_dir, 'cos3d', 3, 'F1000W', nan_thresh=0.4)
produce_cutouts(catalogue, cos3d1_rot, output_dir, 'cos3d', 3, 'F2100W', nan_thresh=0.4)
produce_cutouts(catalogue, cos3d2_rot, output_dir, 'cos3d', 3, 'F1000W', nan_thresh=0.4)
produce_cutouts(catalogue, cos3d2_rot, output_dir, 'cos3d', 3, 'F2100W', nan_thresh=0.4)
